#  SARIMAモデルの構築

- **[4.1 SARIMAモデルの構築(1)](#4.1-)**
    - **[4.1.1 ARIMAモデルの復習](#4.1.1-)**
    - **[4.1.2 sp,sd,sqとは](#4.1.2-)**
    - **[4.1.3 パラメーターの決定](#4.1.3-)**
    - **[4.1.4 自己相関係数・偏自己相関係数とその可視化](#4.1.4-)**
    <br><br>
- **[4.2 SARIMAモデルの構築(2)](#4.2-)**
    - **[4.2.1 モデルの構築](#4.2.1-)**
    - **[4.2.2 予測の実行と予測データの可視化](#4.2.2-)**
    - **[4.2.3 実際データと予測データの比較](#4.2.3-)**
<br><br>
- **[4.3 添削問題](#4.3-添削問題)**

***

## 4.1 SARIMAモデルの構築（１）

### 4.1.1 ARIMAモデルの復習

　前チャプターで説明したARIMAモデルについて、もう一度復習してみましょう。<br>
 ARIMAモデルは、以前の値に影響されるモデルであり、直前p個の値と相関をもつようなARモデル$AR(p)$と、以前の誤差に影響されるモデルで直前q個の値の影響を受けるようなMAモデル$MA(q)$を合成した$ARMA(p, q)$を、d時点前の階差系列に適応したものでした。 <br>
そして、 **SARIMAモデル** とはARIMAモデルをさらに季節周期を持つ時系列データにも拡張できるようにしたモデルです。SARIMAモデルは`(p, d, q)`のパラメーターに加えて`(sp, sd, sq, s)`というパラメーターも持ちます。

#### 問題

- 次の[]に入る言葉を選んでください
- `[]`とはARIMAモデルをさらに季節周期を持つ時系列データにも拡張できるようにしたモデルです。SARIMAモデルは`(p, d, q)`のパラメーターのほかに`(sp, sd, sq, s)`というパラメーターも持ちます。

- SARIMAモデル
- ARMAモデル
- KARIMAモデル
- 上記のすべて

#### ヒント

- SARIMAモデルとはARIMAモデルをさらに季節周期を持つ時系列データにも拡張できるようにしたモデルです。

#### 解答

- SARIMAモデル

***

### 4.1.2 sp,sd,sqとは

$sp,sd,sq$ はそれぞれ **<font color=#AA0000>季節性自己相関</font>** 、 **<font color=#AA0000>季節性導出</font>** 、 **<font color=#AA0000>季節性移動平均</font>** といいます。

ところで、$ARIMA(p, d, q)$ モデルの

・ $p$ は **<font color=#AA0000>自己相関度</font>** といい、 **モデルが直前 $p$ 個の値を用いて予測されるのか** を

・ $d$ は **<font color=#AA0000>誘導</font>** といい、 **時系列データを定常にするために $d$ 次の階差が必要だったこと** を

・ $q$ は **<font color=#AA0000>移動平均</font>** といい、 **モデルが直前 $q$ 個の値に影響を受けること** を


表しているのでした。

 $sp,sd,sq$ も基本的な意味は変わりません。しかし、 $sp,sd,sq$ の場合は、現在のデータはひとつ以上の季節期間を経た過去のデータに影響されます。

例えば12ヶ月周期の季節変動を持つデータの場合、 $s$ のパラメーターは周期を表すため、 $s=12$ となります。

 $sq=1$ ならちょうど12ヶ月(1周期前)のデータ, $sq=2$ なら12ヶ月前のデータと24ヶ月前のデータの影響をモデルが受けることを表します。

分かりにくければ単純に $q$ を $sq$ に置き換えてみてください。

#### 問題

- 次の[]を埋める言葉を選んでください
-  $sq=1$ のときは[]のデータの影響を受けるモデルであることを表す。

- 8ヶ月
- 12ヶ月
- 16ヶ月
- 20ヶ月

#### ヒント

- $sp,sd,sq$ の場合は、現在のデータはひとつ以上の季節期間を経た過去のデータに影響されます。

#### 解答

- 12ヶ月

***

### 4.1.3 パラメーターの決定

PythonにはSARIMAモデルのパラメーター、`(p, d, q)` `(sp, sd, sq, s)`を自動で最も適切にしてくれる機能はありません、そのため **情報量規準** (今回の場合は **BIC(ベイズ情報量基準)** )によってどの値が最も適切なのか調べるプログラムを書かなければなりません。

情報規準量については今回は深く触れませんが、BICの場合は **この値が低ければ低いほどパラメーターの値は適切である** と理解しておいてください。
<br>しかしながら、そのようなコードを書くということ自体は本テーマである時系列分析の趣旨と逸れてしまうため、今回はこちらでそのコードをご用意させていただきました。
時系列データ:`DATA`, パラメータ`s(周期):s`を入力すると、最も良いパラメーターとそのBICを出力します。
```python
def selectparameter(DATA,s):
    p = d = q = range(0, 1)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
    parameters = []
    BICs = np.array([])
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(DATA,
                                            order=param,
                seasonal_order=param_seasonal)
                results = mod.fit()
                parameters.append([param, param_seasonal, results.bic])
                BICs = np.append(y,results.bic)
            except:
                continue
    return print(parameters[np.argmin(BICs)])

```
これを利用して解いて行きましょう。
関数の内容を詳しく言うと各パラメーターがそれぞれ、0,1(今回は単純のためパラメーターの上限を1までとしました)の場合についてのSARIMAモデルのBICを計算し、最もBICが小さくなった場合を表示するようになっています。

ただし、パラメーター $s$ に関しては事前に時系列データや次に説明する偏自己相関の可視化を行うことによって調べておきます。

#### 問題

- 次の最近1年分のスパークリングワインの売上データで最も良いパラメーターは何になるでしょうか、答えてください。(ただし季節変動の周期は12とします。)

In [ ]:
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm

# データの読み込みと整理
sales_sparkring = pd.read_csv(filepath_or_buffer = "https://aidemyexcontentsdata.blob.core.windows.net/data/5060_tsa/monthly-australian-wine-sales-th-sparkling.csv")
index = pd.date_range("1980-01-31","1995-07-31",freq="M")
sales_sparkring.index=index
del sales_sparkring["Month"]

# 1年間分のデータにしています
sales_sparkring = sales_sparkring[:12]

# 関数の定義
def selectparameter(DATA,s):
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], s) for x in list(itertools.product(p, d, q))]
    parameters = []
    BICs = np.array([])
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(DATA,
                                            order=param,
                                            seasonal_order=param_seasonal)
                results = mod.fit()
                parameters.append([param, param_seasonal, results.bic])
                BICs = np.append(BICs,results.bic)
            except:
                continue
    return parameters[np.argmin(BICs)]

# 周期を埋めて最適なパラメーターを求めてください
selectparameter(sales_sparkring,__)

#### ヒント

- 季節変動の周期は12ですから、`s=12`となります


#### 解答例

In [ ]:
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm

# データの読み込みと整理
sales_sparkring = pd.read_csv(filepath_or_buffer = "https://aidemyexcontentsdata.blob.core.windows.net/data/5060_tsa/monthly-australian-wine-sales-th-sparkling.csv")
index = pd.date_range("1980-01-31","1995-07-31",freq="M")
sales_sparkring.index=index
del sales_sparkring["Month"]

# 1年間分のデータにしています
sales_sparkring = sales_sparkring[:12]

# 関数の定義
def selectparameter(DATA,s):
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], s) for x in list(itertools.product(p, d, q))]
    parameters = []
    BICs = np.array([])
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(DATA,
                                            order=param,
                                            seasonal_order=param_seasonal)
                results = mod.fit()
                parameters.append([param, param_seasonal, results.bic])
                BICs = np.append(BICs,results.bic)
            except:
                continue
    return parameters[np.argmin(BICs)]

# 周期を埋めて最適なパラメーターを求めてください
selectparameter(sales_sparkring,12)

### 4.1.4 自己相関係数・偏自己相関係数とその可視化

自己相関とは自己の過去のデータとの相関のことでした。時系列データの分析にはもう一つ、 **<font color=#AA0000>偏自己相関</font>** という値が重要になります。

k次自己相関といえば $y_t$ と $y_{t-k}$ の相関を表しますがk次偏自己相関というと $y_t$ と $y_{t-k}$ の間のデータ、すなわち $y_{t-1}$ から $y_{t-k+1}$ の影響を取り除いた相関になります。

具体的に説明します。7日間差の自己相関係数を求めたとします。しかし、ある日のデータが前日のデータと相関があった場合、7日前→6日前→5日前......1日前→今日とデータを通して相関している可能性があります。この間の影響を取り除いて相関を求めたものが偏自己相関と呼ばれます。
Pythonを用いて自己相関係数と偏自己相関係数を可視化してみましょう。
一般的に月ごとのデータに季節性がある場合にはその周期は12になります。


自己相関係数のグラフは<br>
`sm.graphics.tsa.plot_acf(DATA)`

偏自己相関のグラフは <br>
`sm.graphics.tsa.plot_pacf(DATA)`

で出力できます。

#### 問題

- 前チャプターのセクション1で整えたスパークリングワインの売上データの自己相関係数・偏自己相関係数を可視化してその季節変動の周期を調べてください。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pandas import datetime
%matplotlib inline

# データの読み込みと整理
sales_sparkring = pd.read_csv(filepath_or_buffer = "https://aidemyexcontentsdata.blob.core.windows.net/data/5060_tsa/monthly-australian-wine-sales-th-sparkling.csv")
index = pd.date_range("1980-01-31", "1995-07-31", freq="M")
sales_sparkring.index=index
del sales_sparkring["Month"]
# 自己相関・偏自己相関の可視化
fig=plt.figure(figsize=(12, 8))
# 自己相関係数のグラフを出力します
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(sales_sparkring, lags=30, ax=ax1)
# 偏自己相関係数のグラフを出力します
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(sales_sparkring, lags=30, ax=ax2)
plt.show()
# なにも記入せず実行してください

#### ヒント

- 季節周期を持つデータは周期の部分で偏自己相関が高くなります。スパークリンクワインの売上データが月ごとであることも考慮しましょう。

#### 解答例

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pandas import datetime
%matplotlib inline

# データの読み込みと整理
sales_sparkring = pd.read_csv(filepath_or_buffer = "https://aidemyexcontentsdata.blob.core.windows.net/data/5060_tsa/monthly-australian-wine-sales-th-sparkling.csv")
index = pd.date_range("1980-01-31", "1995-07-31", freq="M")
sales_sparkring.index=index
del sales_sparkring["Month"]
# 自己相関・偏自己相関の可視化
fig=plt.figure(figsize=(12, 8))
# 自己相関係数のグラフを出力します
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(sales_sparkring, lags=30, ax=ax1)
# 偏自己相関係数のグラフを出力します
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(sales_sparkring, lags=30, ax=ax2)
plt.show()
# なにも記入せず実行してください

# 出力結果から周期は12であるとわかります。

## 4.2 SARIMAモデルの構築（２）

### 4.2.1 モデルの構築

ここで一度SARIMAモデルを用いた時系列データの分析手順をまとめてみましょう。

**1.データの読み込み**

**2.データの整理**

**3.データの可視化** 

**4.データの周期の把握 (パラメーターsの決定)** 

**5.s以外のパラメーターの決定**

**6.モデルの構築** 

**7.データとの予測とその可視化** 

という流れで行われます。いよいよ最後にこの節ではモデルの構築から予測までを学びます。
前セクションの方法によって、最適な$(p,d,q),(sp,sd,sq,s)$が分かったらいよいよモデルを構築します。モデルの構築には`sm.tsa.statespace.SARIMAX(DATA,order=(p, d, q),seasonal_order=(sp, sd, sq, s)).fit()`を用います。

#### 問題

- 前セクションで学んだことを踏まえて、スパークリングワインの売上データのSARIMAモデルを構築してください。
- ただし、パラメーター`(p, d, q), (sp, sd, sq, s)`は`(0, 0, 0), (0, 1, 1, 12)`とします。
- 補足:今回のSARIMAモデルのパラメーターが、「4.1.3 パラメーターの決定」で求めたパラメータと違うのは、4.1.3は１年分のデータに対し、今回は15年分(180ヶ月分)のデータを使っているため、パラメーターが違っています。(データ数が多いほど最適になります。)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pandas import datetime
%matplotlib inline
import numpy as np

# データの読み込みと整理
sales_sparkring = pd.read_csv(filepath_or_buffer = "https://aidemyexcontentsdata.blob.core.windows.net/data/5060_tsa/monthly-australian-wine-sales-th-sparkling.csv")
index = pd.date_range("1980-01-31", "1995-07-31", freq="M")
sales_sparkring.index=index
del sales_sparkring["Month"]

# モデルの当てはめ
SARIMA_sparkring_sales = # ここに答えを書き込んでください

#構築したSARIMAモデルのBICを出力します
print(SARIMA_sparkring_sales.bic)

#### ヒント

- モデルの構築には`sm.tsa.statespace.SARIMAX(DATA,order=(p,d,q),seasonal_order=(sp,sd,sq,s)).fit()`を用います。

#### 解答例

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pandas import datetime
%matplotlib inline

# データの読み込みと整理
sales_sparkring = pd.read_csv(filepath_or_buffer = "https://aidemyexcontentsdata.blob.core.windows.net/data/5060_tsa/monthly-australian-wine-sales-th-sparkling.csv")
index = pd.date_range("1980-01-31", "1995-07-31", freq="M")
sales_sparkring.index=index
del sales_sparkring["Month"]

# モデルの当てはめ

SARIMA_sparkring_sales = sm.tsa.statespace.SARIMAX(sales_sparkring,order=(0, 0, 0),seasonal_order=(0, 1, 1, 12)).fit()


#構築したSARIMAモデルのBICを出力します
print(SARIMA_sparkring_sales.bic)

***

### 4.2.2 予測の実行と予測データの可視化

モデルの予測データを得るのは簡単で`モデル名.predict("予測開始時","予測終了時")`を用います。ただし、`"予測開始時"`はもとの時系列データにある時間でなければなりません。例えば、今回使用したスパークリングワインの売上データでは1995-07-31以前である必要があります。

#### 問題

- 前節で作成したスパークリングワインの売上データのSARIMAモデルを用いてモデルの予測を行い、またその予測データを可視化してみましょう。
- 予測データの期間は1994-07-31から1997-12-31までとしてください。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pandas import datetime
%matplotlib inline

# データの読み込みと整理
sales_sparkring = pd.read_csv(filepath_or_buffer = "https://aidemyexcontentsdata.blob.core.windows.net/data/5060_tsa/monthly-australian-wine-sales-th-sparkling.csv")
print(sales_sparkring.head())
print(sales_sparkring.tail())
index = pd.date_range("1980-01-31", "1995-07-31", freq="M")
sales_sparkring.index=index
del sales_sparkring["Month"]

# モデルの当てはめ
SARIMA_sparkring_sales = sm.tsa.statespace.SARIMAX(sales_sparkring,order=(0, 0, 0),seasonal_order=(0, 1, 1, 12)).fit()

# predに予測データを代入する
pred = # ここに答えを書き込んでください

# predデータの可視化
# ここに答えを書き込んでください

#### ヒント

- 予測は`モデル名.predict("予測開始時","予測終了時")`で行います。

#### 解答例

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pandas import datetime
%matplotlib inline

#データの読み込みと整理
sales_sparkring = pd.read_csv(filepath_or_buffer = "https://aidemyexcontentsdata.blob.core.windows.net/data/5060_tsa/monthly-australian-wine-sales-th-sparkling.csv")
print(sales_sparkring.head())
print(sales_sparkring.tail())
index = pd.date_range("1980-01-31", "1995-07-31", freq="M")
sales_sparkring.index=index
del sales_sparkring["Month"]

#モデルの当てはめ
SARIMA_sparkring_sales = sm.tsa.statespace.SARIMAX(sales_sparkring,order=(0, 0, 0),seasonal_order=(0, 1, 1, 12)).fit()

#predに予測データを代入する
pred = SARIMA_sparkring_sales.predict("1994-7-31", "1997-12-31")

#predデータともとの時系列データの可視化
plt.plot(pred)
plt.show()

***

### 4.2.3 実際データと予測データの比較

次は予測データだけを出力するのではなく、もとの時系列データと予測データを同時に出力して比べてみましょう。

#### 問題

- スパークリングワインの売上データと、そのSARIMAモデルによる予測データのグラフを同時に出力して比べてください。
- 予測期間は1994-07-31から1997-12-31までとしてください。また、予測データのグラフは赤で表示させてください。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pandas import datetime
%matplotlib inline
import numpy as np

# データの読み込みと整理
sales_sparkring = pd.read_csv(filepath_or_buffer = "https://aidemyexcontentsdata.blob.core.windows.net/data/5060_tsa/monthly-australian-wine-sales-th-sparkling.csv")
index = pd.date_range("1980-01-31", "1995-07-31", freq="M")
sales_sparkring.index=index
del sales_sparkring["Month"]

# モデルの当てはめ
SARIMA_sparkring_sales = sm.tsa.statespace.SARIMAX(sales_sparkring,order=(0, 0, 0),seasonal_order=(0, 1, 1, 12)).fit()

# predに予測データを代入する
pred = SARIMA_sparkring_sales.predict("1994-7-31", "1997-12-31")

# predデータともとの時系列データの可視化
# ここに答えを入力してください

#### ヒント

- 予想データのグラフは`color="r"`で赤色に指定できます

#### 解答例

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pandas import datetime
%matplotlib inline
import numpy as np

# データの読み込みと整理
sales_sparkring = pd.read_csv(filepath_or_buffer = "https://aidemyexcontentsdata.blob.core.windows.net/data/5060_tsa/monthly-australian-wine-sales-th-sparkling.csv")
index = pd.date_range("1980-01-31", "1995-07-31", freq="M")
sales_sparkring.index=index
del sales_sparkring["Month"]

#モデルの当てはめ
SARIMA_sparkring_sales = sm.tsa.statespace.SARIMAX(sales_sparkring,order=(0, 0, 0),seasonal_order=(0, 1, 1, 12)).fit()

# predに予測データを代入する
pred = SARIMA_sparkring_sales.predict("1994-7-31", "1997-12-31")

# predデータともとの時系列データの可視化
plt.plot(sales_sparkring)
plt.plot(pred, color="r")
plt.show()

***

## 4.3 添削問題

#### 問題

- スパークリングワインの売上分析についてSARIMAモデルを用いて予測して予測データを赤色で元のデータとあわせて出力してください。
- ただし、予測の期間について("開始時","終了時")=("1993-01-31","2050-12-31")としてください。

In [ ]:
import warnings
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pandas import datetime
%matplotlib inline
import numpy as np

#データの読み込みと整理
sales_sparkring = pd.read_csv("monthly-australian-wine-sales-th-sparkling.csv")
index = pd.date_range("1980-01-31","1995-07-31",freq="M")
sales_sparkring.index=index
del sales_sparkring["Month"]

#モデルの当てはめ
#ここに答えを書き込んでください

#predに予測データを代入する
pred = #ここに答えを書き込んでください

#preadデータともとの時系列データの可視化
plt.plot(sales_sparkring)
plt.plot(pred,color="r")
plt.show()

#### ヒント

- モデルの構築にはsm.tsa.statespace.SARIMAX(DATA,order=(p,d,q),seasonal_order=(sp,sd,sq,s).fit()を用います。
- 予測はモデル名.predict("予測開始時","予測終了時")で行います。

#### 解答例

In [ ]:
import warnings
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pandas import datetime
%matplotlib inline
import numpy as np

#データの読み込みと整理
sales_sparkring = pd.read_csv("monthly-australian-wine-sales-th-sparkling.csv")
index = pd.date_range("1980-01-31","1995-07-31",freq="M")
sales_sparkring.index=index
del sales_sparkring["Month"]

#モデルの当てはめ
SARIMA_sparkring_sales = sm.tsa.statespace.SARIMAX(sales_sparkring,order=(0,0,0),seasonal_order=(0, 1, 1, 12)).fit()

#predに予測データを代入する
pred = SARIMA_sparkring_sales.predict("1994-7-31","2050-12-31")

#preadデータともとの時系列データの可視化
plt.plot(sales_sparkring)
plt.plot(pred,color="r")
plt.show()

#### 解説
- `SARIMA_sparkring_sales = sm.tsa.statespace.SARIMAX(sales_sparkring,order=(0,0,0),seasonal_order=(0, 1, 1, 12)).fit()`でモデルを構築しています
- `pred = SARIMA_sparkring_sales.predict("1994-7-31","2050-12-31")`にてpredに予測データを代入して
- `plt.plot(sales_sparkring)
   plt.plot(pred,color="r")`にてデータを可視化しています。
- パラメータについてはこのチャプターの関数を用いて求めたことにしていますが、記入してあっても問題ないです。

***